In [235]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [237]:
Compozit = pd.read_excel('C:/Users/Владимир/OneDrive/Desktop/работа/Обучение/ВКР/Датасет для ВКР_композиты/Compozit_int.xlsx') 

In [6]:
Compozit

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2","Угол нашивки, град",Шаг нашивки,Плотность нашивки
0,1.857143,2030.000000,738.736842,50.000000,23.750000,284.615385,210.000000,70.000000,3000.000000,220.000000,0,4,60
1,1.857143,2030.000000,738.736842,129.000000,21.250000,300.000000,210.000000,70.000000,3000.000000,220.000000,0,5,47
2,2.771331,2030.000000,753.000000,111.860000,22.267857,284.615385,210.000000,70.000000,3000.000000,220.000000,0,5,57
3,2.767918,2000.000000,748.000000,111.860000,22.267857,284.615385,210.000000,70.000000,3000.000000,220.000000,0,5,60
4,2.569620,1910.000000,807.000000,111.860000,22.267857,284.615385,210.000000,70.000000,3000.000000,220.000000,0,5,70
5,2.561475,1900.000000,535.000000,111.860000,22.267857,284.615385,380.000000,75.000000,1800.000000,120.000000,0,7,47
6,3.557018,1930.000000,889.000000,129.000000,21.250000,300.000000,380.000000,75.000000,1800.000000,120.000000,0,7,57
7,3.532338,2100.000000,1421.000000,129.000000,21.250000,300.000000,1010.000000,78.000000,2000.000000,300.000000,0,7,60
8,2.919678,2160.000000,933.000000,129.000000,21.250000,300.000000,1010.000000,78.000000,2000.000000,300.000000,0,7,70
9,2.877358,1990.000000,1628.000000,129.000000,21.250000,300.000000,1010.000000,78.000000,2000.000000,300.000000,0,9,47


In [239]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,GammaRegressor, ARDRegression
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor,BaggingRegressor
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

In [241]:
y_1 = Compozit['Модуль упругости при растяжении, ГПа']
y_2 = Compozit['Прочность при растяжении, МПа']
Compozit.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'], axis=1, inplace=True)
X = Compozit.copy()

Т.к. данные очень сильно отличаются, и они не следуют нормальному распределению, то для будет создано 2 вида для каждой модели. \
Одна из них будет с нормализатором MinMaxScaler

**1. Подбор модели для переменной 'Модуль упругости при растяжении, ГПа'.**

In [243]:
X_train, X_test, y_train, y_test = train_test_split(X, y_1, test_size=0.3, random_state=42)

In [184]:
def my_best_model(X, y_1):
    algos = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}
    },
    'GammaRegressor': {
        'model': GammaRegressor(),
        'params': {}
   },
    'ARDRegression': {
        'model': ARDRegression(),
        'params': {'tol': [1e-3, 1e-2]}
    },
    'AdaBoostRegressor': {
        'model': AdaBoostRegressor(),
        'params': {'n_estimators': [5, 10, 15, 20, 30, 50], 'loss': ['linear', 'square', 'exponential']}
    },
    'BaggingRegressor': {
        'model': BaggingRegressor(),
        'params': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 1.0], 'max_features': [0.5, 1.0]}
    },
    'ExtraTreesRegressor': {
        'model': ExtraTreesRegressor(),
        'params': {'n_estimators': [20, 50, 100, 200], 'max_depth': [2, 5, 10, 20], 'min_samples_split': [2, 5, 10]}
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(),
        'params': {'n_estimators': [20, 50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5], 'max_depth': [3, 5, 7], 'loss': ['ls', 'lad', 'huber', 'quantile']}
    },
    'CatBoostRegressor': {
        'model': CatBoostRegressor(verbose=False),
        'params': {'iterations': [100, 500], 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0], 'subsample': [0.5, 0.7, 1.0], 'depth': [3, 5, 7]}
    }
}

   
    scores = []
    for algo_name, config in algos.items():  # Используем стандартный цикл for
        gs = GridSearchCV(config['model'], config['params'], cv=10, return_train_score=False, scoring='r2')
        gs.fit(X, y_1)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

In [20]:
res_mupr = my_best_model(X_train, y_train)

In [188]:
res_mupr.sort_values(['best_score'])

,model,best_score,best_params
0,LinearRegression,-17.919547,{}
1,GammaRegressor,-14.156777,{}
2,ARDRegression,-6.943522,{'tol': 0.001}
4,BaggingRegressor,-5.173590,"{'max_features': 0.5, 'max_samples': 1.0, 'n_e..."
3,AdaBoostRegressor,-4.036066,"{'loss': 'linear', 'n_estimators': 5}"
5,ExtraTreesRegressor,-3.861599,"{'max_depth': 10, 'min_samples_split': 5, 'n_e..."
6,GradientBoostingRegressor,-3.782408,"{'learning_rate': 0.5, 'loss': 'huber', 'max_d..."
7,CatBoostRegressor,-3.059801,"{'depth': 5, 'iterations': 500, 'learning_rate..."


In [378]:
res_mupr.sort_values(['best_score']).iloc[-1]['best_params']

{'depth': 5, 'iterations': 500, 'learning_rate': 1.0, 'subsample': 0.5}

**1.LinearRegression**

In [26]:
LR =LinearRegression()
LR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(y_test,LR.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, LR.predict(X_test)))
print('R2: ', r2_score(y_test, LR.predict(X_test)))

MAE:  2.0757283830744035
MSE:  5.753046749622899
R2:  0.4012451082682684


In [28]:
LR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', LR)],
                      verbose=True)

In [30]:
LR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', LinearRegression())],
         verbose=True)

In [32]:
print('MAE: ', mean_absolute_error(y_test, LR_norm.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, LR_norm.predict(X_test)))
print('R2: ', r2_score(y_test, LR_norm.predict(X_test)))

MAE:  2.075728383074407
MSE:  5.753046749622944
R2:  0.40124510826826365


**2.GammaRegressor**

In [34]:
GR =GammaRegressor()
GR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(y_test, GR.predict(X_test)))
print('MSE: ', mean_squared_error(y_test,GR.predict(X_test)))
print('R2: ', r2_score(y_test, GR.predict(X_test)))

MAE:  1.841380783077818
MSE:  4.287038768002264
R2:  0.5538215583676274


In [36]:
GR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', GR)],
                      verbose=True)

In [202]:
GR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', GammaRegressor())],
         verbose=True)

In [204]:
print('MAE: ', mean_absolute_error(y_test, GR_norm.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, GR_norm.predict(X_test)))
print('R2: ', r2_score(y_test, GR_norm.predict(X_test)))

MAE:  2.627401594009283
MSE:  9.006257213252807
R2:  0.06266352468232894


**3.ARDRegression**

In [42]:
ARDR =ARDRegression (tol = 0.001)
ARDR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(y_test, ARDR.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, ARDR.predict(X_test)))
print('R2: ', r2_score(y_test, ARDR.predict(X_test)))

MAE:  2.5933364581830367
MSE:  8.303500640478566
R2:  0.1358037152557473


In [44]:
ARDR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ARDR)],
                      verbose=True)

In [46]:
ARDR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', ARDRegression())],
         verbose=True)

In [48]:
print('MAE: ', mean_absolute_error(y_test, ARDR_norm.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, ARDR_norm.predict(X_test)))
print('R2: ', r2_score(y_test, ARDR_norm.predict(X_test)))

MAE:  1.9065296712920816
MSE:  4.279219387907589
R2:  0.5546353692552763


**4.BaggingRegressor**

In [50]:
BR = BaggingRegressor (max_features=0.5, max_samples=1.0, n_estimators=10)
BR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(y_test, BR.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, BR.predict(X_test)))
print('R2: ', r2_score(y_test, BR.predict(X_test)))

MAE:  1.7440505449955002
MSE:  4.936366454795076
R2:  0.48624203994470994


In [52]:
BR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', BR)],
                      verbose=True)

In [54]:
BR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model', BaggingRegressor(max_features=0.5))],
         verbose=True)

In [56]:
print('MAE: ', mean_absolute_error(y_test, BR_norm.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, BR_norm.predict(X_test)))
print('R2: ', r2_score(y_test, BR_norm.predict(X_test)))

MAE:  1.6305257540679332
MSE:  5.921445189580197
R2:  0.3837188488664942


**5.AdaBoostRegressor**

In [245]:
ABR = AdaBoostRegressor (loss= 'linear', n_estimators = 5 )
ABR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(y_test, ABR.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, ABR.predict(X_test)))
print('R2: ', r2_score(y_test, ABR.predict(X_test)))

MAE:  1.3865338439015222
MSE:  3.2038430011787042
R2:  0.6665563912857195


**Сохраним модель**

In [252]:
file_path = "C:/Users/Владимир/ВКР_Итог/Ostroverkh_VKR-2/y_pred_1.pkl"
with open(file_path, "wb") as file:
    pickle.dump(ABR, file)

In [70]:
ABR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ABR)],
                      verbose=True)

In [104]:
ABR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model', AdaBoostRegressor(n_estimators=5))],
         verbose=True)

In [106]:
print('MAE: ', mean_absolute_error(y_test, ABR_norm.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, ABR_norm.predict(X_test)))
print('R2: ', r2_score(y_test, ABR_norm.predict(X_test)))

MAE:  1.3714577629195748
MSE:  3.2364650972278874
R2:  0.6631612094910867


**6.ExtraTreesRegressor**

In [78]:
ETR = ExtraTreesRegressor (max_depth=10, min_samples_split=5, n_estimators=20)
ETR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(y_test, ETR.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, ETR.predict(X_test)))
print('R2: ', r2_score(y_test, ETR.predict(X_test)))

MAE:  1.5670789650148873
MSE:  4.452536794350787
R2:  0.5365971628150238


In [82]:
ETR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ETR)],
                      verbose=True)

In [84]:
ETR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 ExtraTreesRegressor(max_depth=10, min_samples_split=5,
                                     n_estimators=20))],
         verbose=True)

In [86]:
print('MAE: ', mean_absolute_error(y_test, ETR_norm.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, ETR_norm.predict(X_test)))
print('R2: ', r2_score(y_test, ETR_norm.predict(X_test)))

MAE:  1.707833415408017
MSE:  4.880446396893924
R2:  0.49206198364957054


**7.GradientBoostingRegressor**

In [108]:
GBR = GradientBoostingRegressor (learning_rate = 0.5, loss = 'huber', max_depth = 7, n_estimators = 200)
GBR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(y_test, GBR.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, GBR.predict(X_test)))
print('R2: ', r2_score(y_test, GBR.predict(X_test)))

MAE:  1.3397540557530487
MSE:  3.616904150907256
R2:  0.6235665817555753


In [110]:
GBR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', GBR)],
                      verbose=True)

In [114]:
GBR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.3s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 GradientBoostingRegressor(learning_rate=0.5, loss='huber',
                                           max_depth=7, n_estimators=200))],
         verbose=True)

In [116]:
print('MAE: ', mean_absolute_error(y_test, GBR_norm.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, GBR_norm.predict(X_test)))
print('R2: ', r2_score(y_test, GBR_norm.predict(X_test)))

MAE:  1.443435348038939
MSE:  3.845351929408311
R2:  0.5997906190638556


**8.CatBoostRegressor**

In [118]:
CBR = CatBoostRegressor (depth = 5, iterations=500, learning_rate = 1.0, subsample = 0.5, verbose=False)
CBR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(y_test, CBR.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, CBR.predict(X_test)))
print('R2: ', r2_score(y_test, CBR.predict(X_test)))

MAE:  2.0079796660631266
MSE:  6.980367874385071
R2:  0.27351026460039607


In [120]:
CBR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', CBR)],
                      verbose=True)

In [122]:
CBR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.3s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x000001B969A99F70>)],
         verbose=True)

In [124]:
print('MAE: ', mean_absolute_error(y_test, CBR_norm.predict(X_test)))
print('MSE: ', mean_squared_error(y_test, CBR_norm.predict(X_test)))
print('R2: ', r2_score(y_test, CBR_norm.predict(X_test)))

MAE:  2.0079796660631266
MSE:  6.980367874385071
R2:  0.27351026460039607


Наилучший результат показала модель **AdaBoostRegressor**. Результат по оценке **r2 равен 0,67 и 0,66** соответственно\
Файл с моделью будет сохранен и использован в приложении.

**2. Подбор модели для значения 'Прочность при растяжении, МПа'.**

In [126]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y_2, test_size=0.3, random_state=42)

In [415]:
def my_best_model(X, y_2):
    algos = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}
    },
    'GammaRegressor': {
        'model': GammaRegressor(),
        'params': {}
   },
    'ARDRegression': {
        'model': ARDRegression(),
        'params': {'tol': [1e-3, 1e-2]}
    },
    'AdaBoostRegressor': {
        'model': AdaBoostRegressor(),
        'params': {'n_estimators': [5, 10, 15, 20, 30, 50], 'loss': ['linear', 'square', 'exponential']}
    },
    'BaggingRegressor': {
        'model': BaggingRegressor(),
        'params': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 1.0], 'max_features': [0.5, 1.0]}
    },
    'ExtraTreesRegressor': {
        'model': ExtraTreesRegressor(),
        'params': {'n_estimators': [20, 50, 100, 200], 'max_depth': [2, 5, 10, 20], 'min_samples_split': [2, 5, 10]}
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(),
        'params': {'n_estimators': [20, 50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5], 'max_depth': [3, 5, 7], 'loss': ['ls', 'lad', 'huber', 'quantile']}
    },
    'CatBoostRegressor': {
        'model': CatBoostRegressor(verbose=False),
        'params': {'iterations': [100, 500], 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0], 'subsample': [0.5, 0.7, 1.0], 'depth': [3, 5, 7]}
    }
}

   
    scores = []
    for algo_name, config in algos.items():  # Используем стандартный цикл for
        gs = GridSearchCV(config['model'], config['params'], cv=10, return_train_score=False, scoring='r2')
        gs.fit(X, y_2)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

In [417]:
res_mupr = my_best_model(X_train_2, y_train_2)

In [419]:
res_mupr.sort_values(['best_score'])

,model,best_score,best_params
0,LinearRegression,-10.358839,{}
2,ARDRegression,-5.373563,{'tol': 0.01}
6,GradientBoostingRegressor,-3.616197,"{'learning_rate': 0.01, 'loss': 'huber', 'max_..."
1,GammaRegressor,-2.569186,{}
7,CatBoostRegressor,-2.321341,"{'depth': 7, 'iterations': 100, 'learning_rate..."
3,AdaBoostRegressor,-2.243952,"{'loss': 'exponential', 'n_estimators': 30}"
5,ExtraTreesRegressor,-1.972986,"{'max_depth': 10, 'min_samples_split': 2, 'n_e..."
4,BaggingRegressor,-1.668895,"{'max_features': 0.5, 'max_samples': 1.0, 'n_e..."


In [560]:
res_mupr.sort_values(['best_score']).iloc[-1]['best_params']

{'max_features': 0.5, 'max_samples': 1.0, 'n_estimators': 10}

**1.LinearRegression**

In [132]:
LR_2 =LinearRegression()
LR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(y_test_2, LR_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2,LR_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2,LR_2.predict(X_test_2)))

MAE:  623.7745739312428
MSE:  473638.04396031
R2:  -1.0464905525027128


In [226]:
LR_norm_2 = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', LR_2)],
                      verbose=True)

In [228]:
LR_norm_2.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', LinearRegression())],
         verbose=True)

In [230]:
print('MAE: ', mean_absolute_error(y_test_2, LR_norm_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, LR_norm_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2,LR_norm_2.predict(X_test_2)))

MAE:  623.7745739312403
MSE:  473638.0439603066
R2:  -1.0464905525026986


**2.ARDRegression**

In [142]:
ARDR_2 =ARDRegression(tol=0.01)
ARDR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(y_test_2, ARDR_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, ARDR_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, ARDR_2.predict(X_test_2)))

MAE:  449.0353786409726
MSE:  276128.6408880153
R2:  -0.19309388690088314


In [144]:
ARDR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ARDR_2)],
                      verbose=True)

In [146]:
ARDR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', ARDRegression(tol=0.01))],
         verbose=True)

In [148]:
print('MAE: ', mean_absolute_error(y_test_2, ARDR_2_norm.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, ARDR_2_norm.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, ARDR_2_norm.predict(X_test_2)))

MAE:  460.2161313249264
MSE:  278366.25421555666
R2:  -0.20276214432521988


**3.GradientBoostingRegressor**

In [150]:
GBR_2 =GradientBoostingRegressor(learning_rate=0.01, loss='huber', max_depth = 3, n_estimators=20)
GBR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(y_test_2, GBR_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, GBR_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, GBR_2.predict(X_test_2)))

MAE:  431.245184145975
MSE:  243393.78886479465
R2:  -0.05165346365500656


In [152]:
GBR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', GBR_2)],
                      verbose=True)

In [154]:
GBR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 GradientBoostingRegressor(learning_rate=0.01, loss='huber',
                                           n_estimators=20))],
         verbose=True)

In [156]:
print('MAE: ', mean_absolute_error(y_test_2, GBR_2_norm.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, GBR_2_norm.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, GBR_2_norm.predict(X_test_2)))

MAE:  430.2818851570337
MSE:  242669.68628377366
R2:  -0.04852476842032538


**4.GammaRegressor**

In [158]:
GR_2 = GammaRegressor()
GR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(y_test_2, GR_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, GR_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, GR_2.predict(X_test_2)))

MAE:  456.87068777590895
MSE:  262844.88434545905
R2:  -0.13569756366894814


In [162]:
GR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', GR_2)],
                      verbose=True)

In [164]:
GR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', GammaRegressor())],
         verbose=True)

In [166]:
print('MAE: ', mean_absolute_error(y_test_2, GR_2_norm.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, GR_2_norm.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, GR_2_norm.predict(X_test_2)))

MAE:  450.5641697647366
MSE:  257126.75327577573
R2:  -0.11099071977982056


**5.CatBoostRegressor**

In [168]:
CBR_2 =CatBoostRegressor(depth=7, iterations=100, learning_rate=0.1, subsample=0.7,verbose=False)
CBR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(y_test_2, CBR_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, CBR_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, CBR_2.predict(X_test_2)))

MAE:  346.86503609742005
MSE:  173018.99184279112
R2:  0.2524212598101867


In [170]:
CBR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', CBR_2)],
                      verbose=True)

In [172]:
CBR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x000001B969A98AA0>)],
         verbose=True)

In [174]:
print('MAE: ', mean_absolute_error(y_test_2, CBR_2_norm.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, CBR_2_norm.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, CBR_2_norm.predict(X_test_2)))

MAE:  346.86503609742005
MSE:  173018.99184279112
R2:  0.2524212598101867


**6.AdaBoostRegressor**

In [255]:
ABR_2= AdaBoostRegressor (loss= 'exponential', n_estimators=30)
ABR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(y_test_2, ABR_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, ABR_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, ABR_2.predict(X_test_2)))

MAE:  296.21276293646525
MSE:  152585.0966914482
R2:  0.3407118308724185


**Сохраним модель**

In [258]:
file_path = "C:/Users/Владимир/ВКР_Итог/Ostroverkh_VKR-2/y_pred_2.pkl"
with open(file_path, "wb") as file:
    pickle.dump(ABR_2, file)

In [208]:
ABR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ABR_2)],
                      verbose=True)

In [180]:
ABR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 AdaBoostRegressor(loss='exponential', n_estimators=30))],
         verbose=True)

In [182]:
print('MAE: ', mean_absolute_error(y_test_2, ABR_2_norm.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, ABR_2_norm.predict(X_test_2)))
print('R2: ', r2_score(y_test_2,ABR_2_norm.predict(X_test_2)))

MAE:  330.67660579964803
MSE:  159189.23064589096
R2:  0.31217675452546434


**7.ExtraTreesRegressor**

In [184]:
ETR_2 =ExtraTreesRegressor(max_depth=10, min_samples_split = 2, n_estimators=20)
ETR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(y_test_2, ETR_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, ETR_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, ETR_2.predict(X_test_2)))

MAE:  337.3446864582665
MSE:  173246.7967076407
R2:  0.2514369628144667


In [188]:
ETR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ETR_2)],
                      verbose=True)

In [190]:
ETR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model', ExtraTreesRegressor(max_depth=10, n_estimators=20))],
         verbose=True)

In [192]:
print('MAE: ', mean_absolute_error(y_test_2, ETR_2_norm.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, ETR_2_norm.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, ETR_2_norm.predict(X_test_2)))

MAE:  376.45287150348
MSE:  184493.1238302097
R2:  0.2028439443688853


**8.BaggingRegressor**

In [194]:
BR_2 = BaggingRegressor(max_features=0.5, max_samples= 1.0, n_estimators=10)
BR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(y_test_2, BR_2.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, BR_2.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, BR_2.predict(X_test_2)))

MAE:  345.90958467703496
MSE:  180277.1620865965
R2:  0.22106022996510777


In [196]:
BR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', BR_2)],
                      verbose=True)

In [198]:
BR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model', BaggingRegressor(max_features=0.5))],
         verbose=True)

In [200]:
print('MAE: ', mean_absolute_error(y_test_2, BR_2_norm.predict(X_test_2)))
print('MSE: ', mean_squared_error(y_test_2, BR_2_norm.predict(X_test_2)))
print('R2: ', r2_score(y_test_2, BR_2_norm.predict(X_test_2)))

MAE:  343.5622887979627
MSE:  194889.7273682883
R2:  0.15792240314113482


Наилучший результат показала также модель **AdaBoostRegressor**. Результат по оценке **r2 равен 0,34 и 0,31** соответственно\
Файл с моделью будет сохранен и использован в приложении.